In [1]:
!pip install datasets


In [2]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

# Load the dataset (IMDb for sentiment analysis)
dataset = load_dataset('imdb')

# Initialize evaluation metric
metric = evaluate.load('accuracy')

# Define function to compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.argmax(torch.tensor(predictions), axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Define fine-tuning function
def fine_tune_and_evaluate(model_name, tokenizer_class, model_class, is_gpt2=False):
    # Load tokenizer and model
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name, num_labels=2)

    # Handle padding token for GPT-2
    if is_gpt2:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.eos_token_id

    # Tokenize dataset
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Split dataset into train and test sets
    train_dataset = tokenized_datasets['train']
    test_dataset = tokenized_datasets['test']

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        evaluation_strategy='epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate the model
    results = trainer.evaluate()
    print(f"Test accuracy for {model_name}:", results["eval_accuracy"])

# Fine-tune and evaluate BERT
fine_tune_and_evaluate('bert-base-uncased', BertTokenizer, BertForSequenceClassification)

# Fine-tune and evaluate GPT-2
fine_tune_and_evaluate('gpt2', GPT2Tokenizer, GPT2ForSequenceClassification, is_gpt2=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `e

Epoch,Training Loss,Validation Loss
